In [1]:
import numpy as np
import pandas as pd
import spacy as sp

import glob
import os,sys,inspect
import argparse
import re
import ast

import matplotlib.pyplot as plt
import seaborn as sns
from xml.etree import ElementTree

In [2]:
CURRENTDIR = os.path.dirname(os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))))

# Loading textblob extracted data

In [3]:
def read_xml(file_path):
    """Read XML file and return list of tokens
    
    Known structure:
    ---------
    <?xml version="1.0" encoding="utf-8"?>
    <sentiment language="fr" version="1.1" author="Tom De Smedt, Walter Daelemans, fabelier.org" license="PDDL">
        ... List of tokens:
        <word form="abandonné" pos="JJ" polarity="-0.30" subjectivity="0.40" intensity="1.0" confidence="0.9" />
        
    </sentiment>
    
    Input:
    ---------
    file_path: str
        local path, Jupyter cannot access files outside of root, XML file
    
    Output:
    ---------
    tree: xml.etree.ElementTree.Element
    """
    tree = ElementTree.parse(file_path)
    # getroot() gets document, getchildren()[0] gets sample ==> access tokens
    return tree.getroot()

In [8]:
ref_document = read_xml(os.path.join(CURRENTDIR,'data/fr-sentiment.xml'))

In [9]:
def sentiment_to_pandas(document):
    """Select words & tags from xml 
    
    Input:
    -------
    document: xml.etree.ElementTree.Element

    Output:
    -------
    sentence: dataframe
        shape ['form', 'polarity', 'subjectivity', 'intensity', 'confidence']
    """
    vocab = []
    for child in document:
        if child.tag == 'word':
            # first child is solution, if exists
            vocab.append({'form': child.attrib['form'],
                    'polarity': child.attrib['polarity'], 'subjectivity': child.attrib['subjectivity'], 
                    'intensity': child.attrib['intensity'], 'confidence': child.attrib['confidence']
                    })
    return pd.DataFrame(vocab)

In [10]:
pd_vocab = sentiment_to_pandas(ref_document)
pd_vocab.head()

,form,polarity,subjectivity,intensity,confidence
0,abandonné,-0.30,0.40,1.0,0.9
1,abandonnée,-0.30,0.40,1.0,0.8
2,abandonnées,-0.30,0.40,1.0,0.8
3,abandonnés,-0.30,0.40,1.0,0.8
4,abasourdi,0.24,0.55,1.0,0.7


After loading the textblob data inside a dataframe, we load the vocabulary used during the sessions and fuse both dataframes:

In [11]:
results = pd.read_excel(os.path.join(CURRENTDIR,"data/extracted_data.xlsx"))

In [12]:
sentences = '. '.join(results.extract_text.values)
sentences[:1000]

"ah ouais c'est un masque de Batman je sais pas moi j'ai pas vu grand chose d'autre elle était bien belle par contre c'est elle était brillante elle c'était c'était un donc ouais. donc concernant l'image il s'agissait d'une d'une aubergine on aurait dit qu'elle était taillée comme si y avait un masque de Batman et là ah oui oui oui oui elle était elle était propre elle était non ça elle était taillée elle était pas oxydée elle était crue que là quand c'est en aubergine. ah d'accord ok ouais donc on on on a vu exactement la même chose donc là t'as un masque de tortue ninja tout à l'heure c'était Batman hein ouais mais c'est bizarre parce que le le citron enfin le lime le citron vert comme ça enfin je sais pas toi mais moi c'est pas quelque chose que je mange c'est plutôt pour les cocktails que pour que que pour faire une ratatouille quoi donc je sais pas alimentation ouais fruits et légumes c'est aussi une sorte d'. ok donc là il s'agissait d'un demi citron vert avec un masque de tortue

# Spacy POS analysis & word comparison

In [13]:
nlp=sp.load('fr_core_news_sm')

In [14]:
loaded_nlp = nlp(sentences.lower())

In [15]:
def vocab_sentence(loaded_nlp):
    """From a nlp applied to a set of sentences, extract lemmas and pos tags for each token; return unique items
    """
    vocab = []
    for d in loaded_nlp:
        vocab.append((d.lemma_, d.pos_))
    return sorted(list(set(vocab)))

In [16]:
used_vocab = vocab_sentence(loaded_nlp)
print(len(used_vocab))
used_vocab[:20]

2803


[('&', 'CCONJ'),
 ("'", 'PUNCT'),
 ('-', 'PUNCT'),
 ('-fin', 'PUNCT'),
 ('-lors', 'ADV'),
 ('.', 'PUNCT'),
 ('1', 'NUM'),
 ('10', 'NUM'),
 ('2000', 'NUM'),
 ('3', 'NUM'),
 ('4', 'NUM'),
 ('5', 'NUM'),
 ('8000', 'NUM'),
 ('90', 'NUM'),
 ('?', 'PUNCT'),
 ('a.', 'AUX'),
 ('a.', 'VERB'),
 ('abandonner', 'NOUN'),
 ('abandonner', 'VERB'),
 ('abiment', 'VERB')]

Note: some punctuation and split words as well

In [17]:
# extracting lemmas only
u_vocab = [x for x,_ in used_vocab]
# selecting common vocab between spacy and textblob
sel_vocab = pd_vocab[pd_vocab.form.isin(u_vocab)]
sel_vocab.shape

(281, 5)

In [18]:
# vocab in text but not listed in textblob:
rem_vocab = sorted(list(set(u_vocab) - set(pd_vocab[pd_vocab.form.isin(u_vocab)].form.values)))
len(rem_vocab)

1658

In [16]:
sel_vocab.to_excel('polarized_vocab.xlsx', index=False)

In [19]:
for d in rem_vocab:
    print(d, end=' ')

& ' - -fin -lors . 1 10 2000 3 4 5 8000 90 ? a. abandonner abiment abimer abimé abimée abimées abimés abord abricot abîmer accent accord accrobaties accrocher accueil accueillir achat acheter acide adhérer admiration ados adresser adulte affaiblir affaisser afficher afrique agacer agir agriculteur agriculture agrume ah aider ailleurs aimble aimer air alcoolique aliment alimentaire alimentation aller allo allonger allure allusion alors ambiguité ambivalent amener amibe amocher amour amuser améliorer amérique an analyser ananas angle angoisser animation animer anne annoncer année ant antagoniste anti anti-pub antioxydant antithèse antman apelle apparaître apparemment apparence appel appeler appetissante appetissants applatie apple apport apporter approcher apprécier appétissant appêtissant appêtissante après araignée arbre argent argument arnaquer arriver arriérer arrondir arrêter arôme aseptisé asiatique aspect assaisonnement assez association associer attacher attaquer attendre attenti

Non polarised ADJ:

In [20]:
' '.join([x for x,a in used_vocab if (a == 'ADJ' and x not in sel_vocab.form.values)])

"abimé abîmer accrocher acheter agir ah aimble aimer alcoolique alimentaire aller allo amocher amuser amérique animer annoncer appetissante apple apprécier appétissant appêtissant appêtissante asiatique attendre attractif attrayant au auquel automatique autrichien aux auxquels avoir bagarrer bah ballant baraque barbu barrer batman beh ben bio biologique blaser bourrer brillanter briller bruire brut c'est-à-diree ca cabosser cagoule celui-là certes cheveu chimique cibler cifre citron clé cogner coller combattres comestible comment complexer compliquer compléter comprendre concilier connaître consensus contenter contraster cool courber croire croquant crédible deadpool dedans dessiner devoir difformer dire docker domestiquer donald du duck décaler découper décrire défendre déformer défraichie dégoulinant dégueuler déguisement dépressif désoler déterminer empathique endroit enfant engager ensemble entendre entrer envers esthétique euhh exceller excuser expliquer expressif exprimer faillir

Some weird words in this list: "là-bas" "là-dessus" definitely aren't ADJ.

# Marsatag POS analysis & word comparison

In [22]:
marsadir = os.path.join(CURRENTDIR, "convers/marsatag/")

In [23]:
def read_marsa(file_path):
    tree = ElementTree.parse(file_path)
    # getroot() gets document, getchildren()[0] gets sample ==> access tokens
    return tree.getroot().getchildren()[0] 

def marsatag_to_pandas(document, with_inserted=True):
    """Select words & tags from xml 
    
    Input:
    -------
    document: xml.etree.ElementTree.Element
    with_inserted: bool
        whether to remove punctuation inserted by MarsaTag
    
    Output:
    -------
    sentence: dataframe
        shape ['form', 'pos', 'lemma', 'inserted']
    """
    sentence = []
    d = {'A': 'ADJ', 'D':'DET', 'R': 'ADV', 'V': 'VERB', 'C': 'CONJ', 'N': 'NOUN', 
         'S':'PREP', 'W':'PUNCT', 'I':'INTJ', 'P':'PRON', 'U':'X'}
    for child in document:
        if child.tag == 'token':
            # first child is solution, if exists
            try:
                sentence.append({'form': child.attrib['form'], \
                             'pos': d[child.attrib['features'][0]], \
                             'lemma': None if 'lemma' not in child.attrib.keys() else child.attrib['lemma'], \
                             'inserted': (child.attrib['regex_type'] == 'inserted') \
                            })
            except: # erreur de type """<token form="-" regex_type="Ponct_Wm1">"""
                sentence.append({'form': child.attrib['form'], \
                             'pos': 'PUNCT', \
                             'lemma': None if 'lemma' not in child.attrib.keys() else child.attrib['lemma'], \
                             'inserted': (child.attrib['regex_type'] == 'inserted') \
                            })
    if with_inserted:
        return pd.DataFrame(sentence)
    else:
        p = pd.DataFrame(sentence)
        return p[~p.inserted]

Loading POS from MarsaTag requires loading data from XML:

In [24]:
marsa_pd = []
for f in sorted(os.listdir(marsadir)):
    if '.xml' in f:
        marsa_pd.append(marsatag_to_pandas(read_marsa(os.path.join(marsadir, f)), with_inserted=False))
marsa_pd = pd.concat(marsa_pd)
marsa_pd.shape

(82364, 4)

In [25]:
marsa_pd.head()

,form,pos,lemma,inserted
0,ah,INTJ,ah,False
2,ouais,INTJ,ouais,False
4,c',PRON,ce,False
5,est,VERB,être,False
7,un,DET,un,False


Using same format as earlier:

In [26]:
marsa_pd.apply(lambda x: (x.lemma, x.pos), axis=1).values

array([('ah', 'INTJ'), ('ouais', 'INTJ'), ('ce', 'PRON'), ...,
       ('parce_que', 'CONJ'), ('ce', 'PRON'), ('être', 'VERB')],
      dtype=object)

In [27]:
u_marsa = [x for x,_ in list(marsa_pd.apply(lambda x: (x.lemma, x.pos), axis=1).values) if x is not None]
sel_marsa = pd_vocab[pd_vocab.form.isin(u_marsa)]
sel_marsa.shape

(314, 5)

In [28]:
rem_marsa = sorted(list(set(u_marsa) - set(pd_vocab[pd_vocab.form.isin(u_marsa)].form.values)))
len(rem_marsa)

1556

In [31]:
' '.join(sorted(list(set([x for x,a in list(marsa_pd.apply(lambda x: (x.lemma, x.pos), axis=1).values) if (a == 'ADJ' and x not in sel_marsa.form.values and x is not None)]))))

'abandonner abîmer acide affaiblir affaisser aimer alcoolique alimentaire allonger allô ambivalent amoché amuser animer antioxydant appétissant arrondir aseptisé attaquer attractif attrayant automatique autrichien ballant barbu battre bio biologique blaser brunir brut cabossé calibrer casser chimique cinq clean clore cohérent colorer comestible comparer conciliant connaître conservateur considérer contrarier cool croquant crédible cuire dessiner deux disperser disproportionné donner déborder décaler décorer découper défendre défoncer déformer défraîchir dégrader déguiser démoraliser dépité dépressif déprimer dériver désespérer déterminer développer expert expressif fait farfelu fatiguer fiable flasque fondre frapper frit fruitier funky globuleux gélatineux habiller handicapé inaperçu indéchiffrable indéfinissable inhabituel intentionné interdire intuitif issu jeter justifier juteux laitier lassé lever live ludique lustré mangeable marquant masquer mesquin moisir moulant mythique nutrit

In [32]:
sel_marsa.head(40)

,form,polarity,subjectivity,intensity,confidence
21,abordable,0.15,0.30,1.0,0.7
38,absolument,0.25,0.75,2.0,0.9
60,acceptable,0.05,0.10,1.0,0.7
174,affreux,-0.80,0.80,1.0,0.9
190,agressif,-0.80,0.80,1.0,0.9
211,aimable,0.90,0.80,1.0,0.9
239,allemand,0.00,0.00,1.0,1.0
262,aléatoire,-0.05,-0.10,1.0,0.7
272,ambigu,-0.05,1.00,1.0,0.9
291,amusant,0.40,0.75,1.0,0.9


In [32]:
with pd.ExcelWriter('marsa_compare.xlsx') as writer:  
    for f in ['ADJ', 'PREP', 'ADV', 'VERB', 'CONJ', 'DET', 'INTJ', 'NOUN', 'PRON']:
        marsa_pd[marsa_pd.pos == f][['form', 'pos', 'lemma']].drop_duplicates().sort_values(by='form').to_excel(writer, sheet_name=f, index=False)
